In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading


In [7]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [8]:
def scrape_links(page_number):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://jatim.antaranews.com/kabar-jatim/{page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [9]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 9 links from page 1
Total Links: 9


In [10]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            # print(body_elem)
            if body_elem:
                # content_elem = body_elem.find_all('br')
                # print(content_elem)
                content_text = body_elem.text
                # for p in content_elem:
                #     content_text += p.text.strip() + "\n"
                
                # if content_text.strip():
                #     content_text=content_text
                # else:
                #     content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jatim',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [11]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [12]:
df = pd.DataFrame(results)
print(len(results))
df.head(6)

9


,title,author,category,date,content,region,link
0,KPU Sumenep konsultasikan temuan Bawaslu ke KP...,Abd Aziz,Category not found,"Rabu, 27 September 2023 14:47 WIB",\n Madura Raya (ANTARA) - Komisi ...,jateng,https://jatim.antaranews.com/berita/734526/kpu...
1,Mahasiswa UB buat lampu penanda gempa bumi,Endang Sukarelawati,Category not found,"Rabu, 27 September 2023 14:41 WIB",\n Malang Raya (ANTARA) - Empat m...,jateng,https://jatim.antaranews.com/berita/734520/mah...
2,Tokoh Jatim: Judi online marak potret kemiskin...,Abdul Hakim,Category not found,"Rabu, 27 September 2023 11:37 WIB",\n Surabaya (ANTARA) - Tokoh pere...,jateng,https://jatim.antaranews.com/berita/734484/tok...
3,Labkesda Surabaya buka layanan laboratorium ke...,Abdul Hakim,Category not found,"Rabu, 27 September 2023 8:41 WIB",\n Surabaya (ANTARA) - Unit Pelak...,jateng,https://jatim.antaranews.com/berita/734424/lab...
4,"Universitas IBU Malang hadirkan ""Ospek Heppiee...",Endang Sukarelawati,Category not found,"Rabu, 27 September 2023 14:45 WIB",\n Malang Raya (ANTARA) - Univers...,jateng,https://jatim.antaranews.com/berita/734523/uni...
5,Polda Jatim ambil alih kasus karhutla Bromo,Willi Irawan,Category not found,"Rabu, 27 September 2023 10:29 WIB",\n Suraba (ANTARA) - Direktorat R...,jateng,https://jatim.antaranews.com/berita/734463/pol...


In [13]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/jatim_antranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')